In [17]:
#imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
df = pd.read_csv('./locations.csv', sep=',')


In [20]:
#geopy in action
country = 'Turkey'
city_names = df['location']
longitude =[]
latitude =[]
geolocator = Nominatim(user_agent="Trips")

for c in city_names.values:
    location = geolocator.geocode(c + ', ' + country)
    if location is not None:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    else:
        print(f"Koordinatlar bulunamadı for: {c}, {country}")


GeocoderServiceError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)

In [19]:
df['latitude'] = latitude #yeni latitude ve longitude kolonları oluşturuluyor
df['longitude'] = longitude

df

ValueError: Length of values (0) does not match length of index (81)

In [153]:
l2=df.iloc[:,-1:-3:-1] 
l2

,longitude,latitude
0,35.498409,37.143800
1,38.314110,37.789360
2,30.642741,38.685273
3,43.383564,39.529160
4,33.897696,38.432629
...,...,...
76,29.330651,38.609666
77,43.658983,38.324960
78,29.077612,40.592898
79,35.170998,39.715242


In [154]:
kmeans = KMeans(5) 
#kmeans kümeleme algoritmasını seçiyoruz
#kmeans küme merkezlerini rastgele başlatır ve ardından veri noktalarını
# en yakın küme merkezine atar. Bu atama işlemi ve ardından merkezlerin
# yeniden hesaplanması devam eder.
kmeans.fit(l2)
#Sonuç olarak, kmeans.fit(l2) ifadesi, k-means kümeleme modelinin veri kümesi l2 üzerinde eğitilmesini sağlar. Bu, 
# küme merkezlerinin ve veri noktalarının kümelere atanmasının ve modelin eğitilmesinin gerçekleştiği 
# bir işlemdir.
identified_clusters = kmeans.fit_predict(l2)
identified_clusters = list(identified_clusters)
identified_clusters
#veri merkez noktaları 




[3,
 4,
 2,
 1,
 3,
 0,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 1,
 4,
 2,
 1,
 1,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 4,
 4,
 1,
 0,
 3,
 4,
 4,
 1,
 3,
 1,
 2,
 2,
 2,
 3,
 0,
 3,
 1,
 0,
 3,
 3,
 0,
 2,
 0,
 2,
 0,
 2,
 4,
 2,
 1,
 3,
 2,
 1,
 3,
 3,
 4,
 3,
 4,
 2,
 4,
 4,
 1,
 0,
 4,
 1,
 2,
 4,
 4,
 4,
 2,
 1,
 2,
 3,
 0]

In [155]:
df['loc_clusters'] = list(identified_clusters)
df

,location,latitude,longitude,loc_clusters
0,Adana,37.143800,35.498409,3
1,Adıyaman,37.789360,38.314110,4
2,Afyonkarahisar,38.685273,30.642741,2
3,Ağrı,39.529160,43.383564,1
4,Aksaray,38.432629,33.897696,3
...,...,...,...,...
76,Uşak,38.609666,29.330651,2
77,Van,38.324960,43.658983,1
78,Yalova,40.592898,29.077612,2
79,Yozgat,39.715242,35.170998,3


In [172]:
from math import radians, sin, cos, sqrt, atan2
from sklearn.cluster import KMeans
# Haversine fonksiyonu
def haversine(lat1, lon1, lat2, lon2):
    # Radyan cinsinden dönüşümler
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formülü
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = 6371 * c  # Dünya yarıçapı: 6371 km

    return distance

# Input city adı al
input_city = input("Lütfen bir şehir adı girin: ")

# Input city'nin lat ve lon değerlerini al
input_city_data = df[df['location'] == input_city]
input_lat = input_city_data['latitude'].values[0]
input_lon = input_city_data['longitude'].values[0]

# Distance sütununu hesapla ve ekle
distances = []
for index, row in df.iterrows():
    distance = haversine(input_lat, input_lon, row['latitude'], row['longitude'])
    distances.append(distance)

df['distance'] = distances


# Input city'nin lat ve lon değerlerini al
input_city_data = df[df['location'] == input_city]
input_lat = input_city_data['latitude'].values[0]
input_lon = input_city_data['longitude'].values[0]

# Distance sütununu hesapla ve ekle
distances = []
for index, row in df.iterrows():
    distance = haversine(input_lat, input_lon, row['latitude'], row['longitude'])
    distances.append(distance)

df['distance'] = distances

# K-Means kümeleme modelini oluştur
kmeans = KMeans(n_clusters=7, random_state=0)

# Verileri kümelere ayır
df['kmeans_clusters'] = kmeans.fit_predict(df[['distance']])
cluster = df.loc[df['location'] == input_city, 'kmeans_clusters']
cluster = cluster.iloc[0]
cities = df.loc[df['kmeans_clusters'] == cluster, 'location']

for c in range(len(cities)):
     if cities.iloc[c] == input_city:
         continue
     else:
         print(cities.iloc[c])




print("sa",df)


Balıkesir
Bilecik
Bolu
Bursa
Çanakkale
Düzce
Edirne
Eskişehir
Kırklareli
Kocaeli
Kütahya
Manisa
Sakarya
Tekirdağ
Yalova
Zonguldak


,location,latitude,longitude,loc_clusters,distance,kmeans_clusters
0,Adana,37.143800,35.498409,3,707.809990,6
1,Adıyaman,37.789360,38.314110,4,877.910258,3
2,Afyonkarahisar,38.685273,30.642741,2,294.711166,4
3,Ağrı,39.529160,43.383564,1,1231.967492,0
4,Aksaray,38.432629,33.897696,3,508.895368,2
...,...,...,...,...,...,...
76,Uşak,38.609666,29.330651,2,268.219286,4
77,Van,38.324960,43.658983,1,1289.986116,0
78,Yalova,40.592898,29.077612,2,46.768022,1
79,Yozgat,39.715242,35.170998,3,544.045956,2
